In [1]:
%matplotlib notebook
from pyntcloud import PyntCloud
import numpy as np
import pandas as pd
import cv2
import math
import random
import matplotlib.pyplot as plt # matplotlibの描画系

In [2]:
#同次座標系に関する関数
def makeHomogeneous(vector):
    rows,cols = vector.shape[:2]
    ones = [np.ones(cols)]
    return np.r_[vector, ones]
    
def delHomogeneous(vector):
    rows,cols = vector.shape[:2]
    val = vector[:rows-1]
    dim = vector[rows-1:]
    return val / dim

def generateM(R, t):
    M = np.eye(4)
    M[:3,:3] = R
    M[:3,3] = t.T
    return M

def invM(M):
    R = M[:3,:3].T
    t = M[:3,3]
    Rt = -R.dot(t)
    newM = np.eye((4))
    newM[:3,:3] = R
    newM[0:3,3] = Rt.T
    return newM

def sampling3d(point3d, samples):
    indexes = np.random.randint(0, point3d.shape[0], samples)
    return point3d[indexes]

#ワールド座標系の三次元位置をカメラ位置での正規化座標系に投影する
def get_proj_pos(vec3d_w,M):
    h_vec3d_w = makeHomogeneous(vec3d_w)
    vec3d_c = M.dot(h_vec3d_w)
    vec2d = vec3d_c[0:2,:]
    vec_z = vec3d_c[2,:]
    vec2d = vec2d / vec_z
    return vec2d

#三次元回転行列生成に関する関数
    # 物体座標系の 1->2->3 軸で回転させる
def generateRotationMatrix(r):
    px = r[0]
    py = r[1]
    pz = r[2]
    
    Rx = np.array([[1, 0, 0],
            [0, np.cos(px), np.sin(px)],
            [0, -np.sin(px), np.cos(px)]])
    Ry = np.array([[np.cos(py), 0, -np.sin(py)],
            [0, 1, 0],
            [np.sin(py), 0, np.cos(py)]])
    Rz = np.array([[np.cos(pz), np.sin(pz), 0],
            [-np.sin(pz), np.cos(pz), 0],
            [0, 0, 1]])
    R = Rz.dot(Ry).dot(Rx)
    return R

def generateTransMatrix(t, r):
    R = generateRotationMatrix(r)
    transMatrix = np.eye(4,4)
    for y in range(0,3):
        for x in range(0,3):
            transMatrix[y, x] = R[y, x]
        transMatrix[y, 3] = t[y]
    return transMatrix

def generatePyntCloud(point3d):
    """numpy形式の3d点群からpyntcloud用データを生成する。
       @param[in] point3d [[x,y,z],...] 
    """
    w_color = False
    if(point3d.shape[1] == 6):
        w_color = True
    points = pd.DataFrame(point3d)

    if(w_color == False):
        points.columns = ['x', 'y', 'z']
    else:
        points.columns = ['x', 'y', 'z', 'blue', 'green', 'red']
    cloud = PyntCloud(points)
    return cloud

def axisPyntCloud(scale):
    # color, [vertexes]
    lines = {
        '0xFF0000': [[0, 0, 0], [scale, 0, 0]],
        '0x00FF00': [[0, 0, 0], [0, scale, 0]],
        '0x0000FF': [[0, 0, 0], [0, 0, scale]]
    }
    return lines

def addColor(point3d, color):
    return np.concatenate((point3d, np.ones(point3d.shape) * np.array(color)) ,axis=1)

def warp3d(point3d, M):
    h_point3d = makeHomogeneous(point3d)
    h_w_point3d = M.dot(h_point3d)
    return delHomogeneous(h_w_point3d)

In [3]:
#bunnyのデータを読み込み、重心中心に移動する
cloud = PyntCloud.from_file("data/stanford/bun_zipper.ply")
point3d = sampling3d(cloud.get_mesh_vertices()[0], 1000) 
point3d = point3d - np.mean(point3d, axis = 0)
cloud = generatePyntCloud(addColor(point3d, [0, 255, 0]))
cloud.plot(point_size=0.002, opacity=1.0, polylines=axisPyntCloud(0.1), backend="threejs")

In [4]:
#モデルデータ生成
obj = point3d.T.copy()

noise = np.random.normal(0, 0.005, obj.shape)

obs = obj + noise

cloud = generatePyntCloud(np.concatenate((addColor(obs.T, [0, 255, 255]), addColor(obj.T,[0, 255, 0]))))
cloud.plot(point_size=0.002, opacity=1.0, polylines=axisPyntCloud(0.1), backend="threejs")

In [5]:
g_M = []

#カメラ位置を生成
for i in range(10):
    #y軸回り
    r = i * math.pi / 180 * 10
    scale = 0.2
    t = np.array([math.cos(r) - math.sin(r), 0, math.sin(r) + math.cos(r)])\
        * scale
    p = np.array([0, r + math.pi * 3 / 4, 0])
    dM = generateTransMatrix(t, p)
    g_M.append(dM)

In [6]:
def addCamera(pose, line = {}, scale = 0.01, color = '0xFFFFFF'):
    # color, [vertexes]
    frustum = [[0, 0, 0], [scale * 1.3, scale, scale], [scale * 1.3, -scale, scale],\
           [0, 0, 0], [scale * 1.3, -scale, scale], [-scale * 1.3, -scale, scale],\
           [0, 0, 0], [-scale * 1.3, -scale, scale], [-scale * 1.3, scale, scale],\
           [0, 0, 0], [-scale * 1.3, scale, scale], [scale * 1.3, scale, scale]]
    
    for i, f in enumerate(frustum):
        vector = np.array(f).reshape([3,1])
        frustum[i] = delHomogeneous(pose.dot(makeHomogeneous(vector))).reshape(-1).tolist()
    cam = {
        color: frustum
    }
    line.update(cam)
    return line

line = axisPyntCloud(0.1)
color = '0xFFFFFF'
for i, m in enumerate(g_M):
    line = addCamera(m, line, 0.01, color + str(i))

cloud = generatePyntCloud(np.concatenate((addColor(obs.T, [0, 255, 255]), addColor(obj.T,[0, 255, 0]))))
cloud.plot(point_size=0.002, opacity=1.0, polylines=line, backend="threejs")

In [7]:
norm_K = np.eye(3,3)
norm_d = np.zeros(5)

#カメラ数等の値を算出する
num_camera = len(g_M)
num_obj = obj.shape[1]
num_fix_camera = 2

#ノイズを乗せたカメラ位置を作成する
mu = 0.0
sigma = 0.01
obs_M = []#観測したカメラ位置
count = 0
for mat in g_M:
    if count < num_fix_camera:
        err = np.array([[0,0,0]]).reshape(3,1)
    else:
        err = np.array((random.gauss(mu, sigma), random.gauss(mu, sigma),random.gauss(mu, sigma))).reshape(3,1)
    obs_mat = mat.copy()
    obs_mat[0:3,3:4] += err
    obs_M.append(obs_mat)
    count += 1
    
line = axisPyntCloud(0.1)
color = '0xFFFFFF'
for i, m in enumerate(g_M):
    line = addCamera(m, line, 0.01, color + str(i))

color = '0x00FFFF'
for i, m in enumerate(obs_M):
    line = addCamera(m, line, 0.01, color + str(i + num_camera))
    
cloud = generatePyntCloud(np.concatenate((addColor(obs.T, [0, 255, 255]), addColor(obj.T,[0, 255, 0]))))
cloud.plot(point_size=0.002, opacity=1.0, polylines=line, backend="threejs")

In [8]:
#推定した三次元位置、カメラ位置を投影する
norm_img_points = []
for i in range(num_camera):
    tmp_cam_num = i
    pos2d = get_proj_pos(obj, g_M[i])
    norm_img_points.append(pos2d.T)
    obs_pos2d = get_proj_pos(obs, obs_M[i])
    #plt.figure()
    #plt.scatter(pos2d[0,:],pos2d[1,:], color="#00ff00", s=1)
    #plt.scatter(obs_pos2d[0,:],obs_pos2d[1,:], color="#0088ff", s=1)
    #plt.show()

### バンドル調整

In [9]:
param_num_camera = 6 * (num_camera - num_fix_camera)
param_num_obj = 3 * num_obj
param_num = param_num_camera + param_num_obj
num_loop = 10

print ("num_camera= " + str(num_camera))
print ("num_obj= " + str(num_obj))
print ("param_camera= " + str(param_num_camera))
print ("param_obj= " + str(param_num_obj))

num_camera= 10
num_obj= 1000
param_camera= 48
param_obj= 3000


In [10]:
import copy
est_M = copy.deepcopy(obs_M)
est_obj = obs.copy()

lamda = 0.0

for l in range(num_loop):
    h_obj = makeHomogeneous(est_obj)
    JtJ = np.zeros((param_num, param_num),np.float64)
    JtE = np.zeros((param_num, 1),np.float64)
    
    saved_M = copy.deepcopy(est_M)
    saved_obj = est_obj.copy()
    
    sum_err = 0.0
    #各カメラに対してのヤコビアンを計算する
    for cam in range(num_camera):
        #カメラ座標系におけるobjの位置
        vec_Xc = est_M[cam].dot(h_obj)
        p_camera = 6 * (cam - num_fix_camera)
        R = est_M[cam][0:3,0:3]
        
        #各objに対してヤコビアンを計算する
        for o in range(num_obj):            
            p_obj = param_num_camera + 3 * o
            Xc = vec_Xc[:,o]
            
            invZ = 1.0 / Xc[2]
            x = Xc[0] * invZ
            y = Xc[1] * invZ
            xy = x*y
            
            est_x = norm_img_points[cam][o,0]
            est_y = norm_img_points[cam][o,1]
            dx = est_x - x
            dy = est_y - y
            
            sum_err += math.sqrt(dx*dx + dy*dy)
            
            b = np.array([[invZ,
                          0.0,
                          -x * invZ,
                          -xy,
                          1.0 + x*x,
                          -y]])

            c = np.array([[0.0,
                          invZ,
                          -y * invZ,
                          -1.0 - y*y,
                          xy,
                          x]])
            
            A = b.T * b + c.T * c
            E_A =b.T * dx + c.T * dy
                        
            a = np.array([[R[0,0] - R[2,0]*x,
                           R[0,1] - R[2,1]*x,
                           R[0,2] - R[2,2]*x,
                           R[1,0] - R[2,0]*y,
                           R[1,1] - R[2,1]*y,
                           R[1,2] - R[2,2]*y]]) * invZ
            
            d = a[:,0:3]
            e = a[:,3:6]
                          
            B = b.T * d + c.T * e
            C = d.T * d + e.T * e
                        
            E_B = d.T*dx + e.T*dy
            
            JtJ[p_obj: p_obj+3, p_obj: p_obj+3] += C
            JtE[p_obj: p_obj+3] += E_B

            #カメラ0は固定する
            if cam < num_fix_camera:
                continue
                
            JtJ[p_camera: p_camera+6, p_camera: p_camera+6] += A
                        
            JtE[p_camera: p_camera+6] += E_A
            JtJ[p_camera: p_camera+6, p_obj: p_obj+3] += B
            JtJ[p_obj: p_obj+3, p_camera: p_camera+6] += B.T
    
    #plt.imshow(JtJ)
    #plt.show() 
    
    if(l==0):
        lamda = JtJ.trace() / (param_num * 1000)
        
    for i in range(param_num):
        JtJ[i,i]+=lamda
    
    delta = np.linalg.solve(JtJ, JtE)
    
    #update_parameter
    for cam in range(num_camera - num_fix_camera):
        p_camera = 6*cam
        dt = delta[p_camera:p_camera+3,:]
        omega = delta[p_camera+3:p_camera+6,:]

        #補正量計算
        angle = math.sqrt(omega[0]*omega[0] + omega[1]*omega[1] + omega[2]*omega[2])
        dR = np.eye(3,3)
        if angle < 1.0e-12:
            pass
        else:
            dR[0,1] = -omega[2]
            dR[0,2] = omega[1]
            dR[1,0] = omega[2]
            dR[1,2] = -omega[0]
            dR[2,0] = -omega[1]
            dR[2,1] = omega[0]

        delta_M = generateM(dR,dt)
        est_M[cam + num_fix_camera] = delta_M.dot(est_M[cam + num_fix_camera])

    for o in range(num_obj):
        p_obj = param_num_camera + 3 * o
        dt = delta[p_obj:p_obj+3,:].reshape(3)
        est_obj[:,o] += dt

    cur_err = 0.0
    #各カメラに対してのヤコビアンを計算する
    for cam in range(num_camera):
        #カメラ座標系におけるobjの位置
        vec_Xc = est_M[cam].dot(h_obj)
        p_camera = 6 * cam
        R = est_M[cam][0:3,0:3]
        
        #各objに対してヤコビアンを計算する
        for o in range(num_obj):            
            p_obj = param_num_camera + 3 * o
            
            Xc = vec_Xc[:,o]
            
            invZ = 1.0 / Xc[2]
            x = Xc[0] * invZ
            y = Xc[1] * invZ
            xy = x*y
            
            est_x = norm_img_points[cam][o,0]
            est_y = norm_img_points[cam][o,1]
            dx = est_x - x
            dy = est_y - y
            
            cur_err += math.sqrt(dx*dx + dy*dy)
   
    if cur_err > sum_err:
        lamda *= 10.0
        est_M = copy.deepcopy(saved_M)
        est_obj = saved_obj.copy()

    print ('loop:', l, ' error:',sum_err)

loop: 0  error: 463.9952438258737
loop: 1  error: 11.982938870923654
loop: 2  error: 0.057078117387426824
loop: 3  error: 0.0013579554215451856
loop: 4  error: 0.0013579554215451856
loop: 5  error: 0.0013579554215451856
loop: 6  error: 0.0012405984210377076
loop: 7  error: 0.0011466755289797064
loop: 8  error: 0.0010692072526209367
loop: 9  error: 0.0010043223474466955


In [12]:
line = axisPyntCloud(0.1)
color = '0xFFFFFF'
for i, m in enumerate(g_M):
    line = addCamera(m, line, 0.01, color + str(i))

color = '0x00FFFF'
for i, m in enumerate(est_M):
    line = addCamera(m, line, 0.01, color + str(i + num_camera))
    
cloud = generatePyntCloud(np.concatenate((addColor(est_obj.T, [0, 255, 255]), addColor(obj.T,[0, 255, 0]))))
cloud.plot(point_size=0.002, opacity=1.0, polylines=line, backend="threejs")

 * Copyright (c) 2018-2021, tmako123
 * https://twitter.com/tmako123
 * http://blog.livedoor.jp/tmako123-programming/
 * All rights reserved.
 * This file is distributed under the GNU Lesser General Public License v3.0.
 * The complete license agreement can be obtained at:
 * http://www.gnu.org/licenses/lgpl-3.0.html